In [3]:
"""Training script for the WaveNet network on the VCTK corpus.

This script trains a network with the WaveNet using data from the VCTK corpus,
which can be freely downloaded at the following site (~10 GB):
http://homepages.inf.ed.ac.uk/jyamagis/page3/page58/page58.html
"""

from __future__ import print_function

import argparse
from datetime import datetime
import json
import os
import sys
import time

import tensorflow as tf
from tensorflow.python.client import timeline

from wavenet import WaveNetModel, AudioReader, optimizer_factory

In [4]:
BATCH_SIZE = 1
DATA_DIRECTORY = './VCTK-Corpus'
LOGDIR_ROOT = './logdir'
CHECKPOINT_EVERY = 50
NUM_STEPS = int(1e5)
LEARNING_RATE = 1e-3
WAVENET_PARAMS = './wavenet_params.json'
STARTED_DATESTRING = "{0:%Y-%m-%dT%H-%M-%S}".format(datetime.now())
SAMPLE_SIZE = 100000
L2_REGULARIZATION_STRENGTH = 0
#SILENCE_THRESHOLD = 0.3
SILENCE_THRESHOLD = 0
EPSILON = 0.001
MOMENTUM = 0.9
MAX_TO_KEEP = 5
METADATA = False

In [5]:
def get_arguments():
    def _str_to_bool(s):
        """Convert string to bool (in argparse context)."""
        if s.lower() not in ['true', 'false']:
            raise ValueError('Argument needs to be a '
                             'boolean, got {}'.format(s))
        return {'true': True, 'false': False}[s.lower()]

    parser = argparse.ArgumentParser(description='WaveNet example network')
    parser.add_argument('--batch_size', type=int, default=BATCH_SIZE,
                        help='How many wav files to process at once. Default: ' + str(BATCH_SIZE) + '.')
    parser.add_argument('--data_dir', type=str, default=DATA_DIRECTORY,
                        help='The directory containing the VCTK corpus.')
    parser.add_argument('--store_metadata', type=bool, default=METADATA,
                        help='Whether to store advanced debugging information '
                        '(execution time, memory consumption) for use with '
                        'TensorBoard. Default: ' + str(METADATA) + '.')
    parser.add_argument('--logdir', type=str, default=None,
                        help='Directory in which to store the logging '
                        'information for TensorBoard. '
                        'If the model already exists, it will restore '
                        'the state and will continue training. '
                        'Cannot use with --logdir_root and --restore_from.')
    parser.add_argument('--logdir_root', type=str, default=None,
                        help='Root directory to place the logging '
                        'output and generated model. These are stored '
                        'under the dated subdirectory of --logdir_root. '
                        'Cannot use with --logdir.')
    parser.add_argument('--restore_from', type=str, default=None,
                        help='Directory in which to restore the model from. '
                        'This creates the new model under the dated directory '
                        'in --logdir_root. '
                        'Cannot use with --logdir.')
    parser.add_argument('--checkpoint_every', type=int,
                        default=CHECKPOINT_EVERY,
                        help='How many steps to save each checkpoint after. Default: ' + str(CHECKPOINT_EVERY) + '.')
    parser.add_argument('--num_steps', type=int, default=NUM_STEPS,
                        help='Number of training steps. Default: ' + str(NUM_STEPS) + '.')
    parser.add_argument('--learning_rate', type=float, default=LEARNING_RATE,
                        help='Learning rate for training. Default: ' + str(LEARNING_RATE) + '.')
    parser.add_argument('--wavenet_params', type=str, default=WAVENET_PARAMS,
                        help='JSON file with the network parameters. Default: ' + WAVENET_PARAMS + '.')
    parser.add_argument('--sample_size', type=int, default=SAMPLE_SIZE,
                        help='Concatenate and cut audio samples to this many '
                        'samples. Default: ' + str(SAMPLE_SIZE) + '.')
    parser.add_argument('--l2_regularization_strength', type=float,
                        default=L2_REGULARIZATION_STRENGTH,
                        help='Coefficient in the L2 regularization. '
                        'Default: False')
    parser.add_argument('--silence_threshold', type=float,
                        default=SILENCE_THRESHOLD,
                        help='Volume threshold below which to trim the start '
                        'and the end from the training set samples. Default: ' + str(SILENCE_THRESHOLD) + '.')
    parser.add_argument('--optimizer', type=str, default='adam',
                        choices=optimizer_factory.keys(),
                        help='Select the optimizer specified by this option. Default: adam.')
    parser.add_argument('--momentum', type=float,
                        default=MOMENTUM, help='Specify the momentum to be '
                        'used by sgd or rmsprop optimizer. Ignored by the '
                        'adam optimizer. Default: ' + str(MOMENTUM) + '.')
    parser.add_argument('--histograms', type=_str_to_bool, default=False,
                        help='Whether to store histogram summaries. Default: False')
    parser.add_argument('--gc_channels', type=int, default=None,
                        help='Number of global condition channels. Default: None. Expecting: Int')
    parser.add_argument('--max_checkpoints', type=int, default=MAX_TO_KEEP,
                        help='Maximum amount of checkpoints that will be kept alive. Default: '
                             + str(MAX_TO_KEEP) + '.')
    return parser.parse_args([])


In [6]:
def save(saver, sess, logdir, step):
    model_name = 'model.ckpt'
    checkpoint_path = os.path.join(logdir, model_name)
    print('Storing checkpoint to {} ...'.format(logdir), end="")
    sys.stdout.flush()

    if not os.path.exists(logdir):
        os.makedirs(logdir)

    saver.save(sess, checkpoint_path, global_step=step)
    print(' Done.')

In [7]:
def load(saver, sess, logdir):
    print("Trying to restore saved checkpoints from {} ...".format(logdir),
          end="")

    ckpt = tf.train.get_checkpoint_state(logdir)
    if ckpt:
        print("  Checkpoint found: {}".format(ckpt.model_checkpoint_path))
        global_step = int(ckpt.model_checkpoint_path
                          .split('/')[-1]
                          .split('-')[-1])
        print("  Global step was: {}".format(global_step))
        print("  Restoring...", end="")
        saver.restore(sess, ckpt.model_checkpoint_path)
        print(" Done.")
        return global_step
    else:
        print(" No checkpoint found.")
        return None

In [8]:
def get_default_logdir(logdir_root):
    logdir = os.path.join(logdir_root, 'train', STARTED_DATESTRING)
    return logdir


def validate_directories(args):
    """Validate and arrange directory related arguments."""

    # Validation
    if args.logdir and args.logdir_root:
        raise ValueError("--logdir and --logdir_root cannot be "
                         "specified at the same time.")

    if args.logdir and args.restore_from:
        raise ValueError(
            "--logdir and --restore_from cannot be specified at the same "
            "time. This is to keep your previous model from unexpected "
            "overwrites.\n"
            "Use --logdir_root to specify the root of the directory which "
            "will be automatically created with current date and time, or use "
            "only --logdir to just continue the training from the last "
            "checkpoint.")

    # Arrangement
    logdir_root = args.logdir_root
    if logdir_root is None:
        logdir_root = LOGDIR_ROOT

    logdir = args.logdir
    if logdir is None:
        logdir = get_default_logdir(logdir_root)
        print('Using default logdir: {}'.format(logdir))

    restore_from = args.restore_from
    if restore_from is None:
        # args.logdir and args.restore_from are exclusive,
        # so it is guaranteed the logdir here is newly created.
        restore_from = logdir

    return {
        'logdir': logdir,
        'logdir_root': args.logdir_root,
        'restore_from': restore_from
    }


In [ ]:
args = get_arguments()

try:
    directories = validate_directories(args)
except ValueError as e:
    print("Some arguments are wrong:")
    print(str(e))

logdir = directories['logdir']
restore_from = directories['restore_from']

# Even if we restored the model, we will treat it as new training
# if the trained model is written into an arbitrary location.
is_overwritten_training = logdir != restore_from

with open(args.wavenet_params, 'r') as f:
    wavenet_params = json.load(f)

# Create coordinator.
coord = tf.train.Coordinator()

# Load raw waveform from VCTK corpus.
with tf.name_scope('create_inputs'):
    # Allow silence trimming to be skipped by specifying a threshold near
    # zero.
    silence_threshold = args.silence_threshold if args.silence_threshold > \
                                                  EPSILON else None
    gc_enabled = args.gc_channels is not None
    reader = AudioReader(
        args.data_dir,
        coord,
        sample_rate=wavenet_params['sample_rate'],
        gc_enabled=gc_enabled,
        receptive_field=WaveNetModel.calculate_receptive_field(wavenet_params["filter_width"],
                                                               wavenet_params["dilations"],
                                                               wavenet_params["scalar_input"],
                                                               wavenet_params["initial_filter_width"]),
        sample_size=args.sample_size,
        silence_threshold=silence_threshold)
    audio_batch = reader.dequeue(args.batch_size)
    if gc_enabled:
        gc_id_batch = reader.dequeue_gc(args.batch_size)
    else:
        gc_id_batch = None

# Create network.
net = WaveNetModel(
    batch_size=args.batch_size,
    dilations=wavenet_params["dilations"],
    filter_width=wavenet_params["filter_width"],
    residual_channels=wavenet_params["residual_channels"],
    dilation_channels=wavenet_params["dilation_channels"],
    skip_channels=wavenet_params["skip_channels"],
    quantization_channels=wavenet_params["quantization_channels"],
    use_biases=wavenet_params["use_biases"],
    scalar_input=wavenet_params["scalar_input"],
    initial_filter_width=wavenet_params["initial_filter_width"],
    histograms=args.histograms,
    global_condition_channels=args.gc_channels,
    global_condition_cardinality=reader.gc_category_cardinality)

if args.l2_regularization_strength == 0:
    args.l2_regularization_strength = None
loss = net.loss(input_batch=audio_batch,
                global_condition_batch=gc_id_batch,
                l2_regularization_strength=args.l2_regularization_strength)
optimizer = optimizer_factory[args.optimizer](
                learning_rate=args.learning_rate,
                momentum=args.momentum)
trainable = tf.trainable_variables()
optim = optimizer.minimize(loss, var_list=trainable)

# Set up logging for TensorBoard.
writer = tf.summary.FileWriter(logdir)
writer.add_graph(tf.get_default_graph())
run_metadata = tf.RunMetadata()
summaries = tf.summary.merge_all()

# Set up session
sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
init = tf.global_variables_initializer()
sess.run(init)

# Saver for storing checkpoints of the model.
saver = tf.train.Saver(var_list=tf.trainable_variables(), max_to_keep=args.max_checkpoints)

try:
    saved_global_step = load(saver, sess, restore_from)
    if is_overwritten_training or saved_global_step is None:
        # The first training step will be saved_global_step + 1,
        # therefore we put -1 here for new or overwritten trainings.
        saved_global_step = -1

except:
    print("Something went wrong while restoring checkpoint. "
          "We will terminate training to avoid accidentally overwriting "
          "the previous model.")
    raise

threads = tf.train.start_queue_runners(sess=sess, coord=coord)
reader.start_threads(sess)

step = None
last_saved_step = saved_global_step
try:
    for step in range(saved_global_step + 1, args.num_steps):
        start_time = time.time()
        if args.store_metadata and step % 50 == 0:
            # Slow run that stores extra information for debugging.
            print('Storing metadata')
            run_options = tf.RunOptions(
                trace_level=tf.RunOptions.FULL_TRACE)
            summary, loss_value, _ = sess.run(
                [summaries, loss, optim],
                options=run_options,
                run_metadata=run_metadata)
            writer.add_summary(summary, step)
            writer.add_run_metadata(run_metadata,
                                    'step_{:04d}'.format(step))
            tl = timeline.Timeline(run_metadata.step_stats)
            timeline_path = os.path.join(logdir, 'timeline.trace')
            with open(timeline_path, 'w') as f:
                f.write(tl.generate_chrome_trace_format(show_memory=True))
        else:
            summary, loss_value, _ = sess.run([summaries, loss, optim])
            writer.add_summary(summary, step)

        duration = time.time() - start_time
        print('step {:d} - loss = {:.3f}, ({:.3f} sec/step)'
              .format(step, loss_value, duration))

        if step % args.checkpoint_every == 0:
            save(saver, sess, logdir, step)
            last_saved_step = step

except KeyboardInterrupt:
    # Introduce a line break after ^C is displayed so save message
    # is on its own line.
    print()
finally:
    if step > last_saved_step:
        save(saver, sess, logdir, step)
    coord.request_stop()
    coord.join(threads)




Using default logdir: ./logdir/train/2018-05-31T10-54-40
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Trying to restore saved checkpoints from ./logdir/train/2018-05-31T10-54-40 ... No checkpoint found.
files length: 44257
step 0 - loss = 5.661, (17.984 sec/step)
Storing checkpoint to ./logdir/train/2018-05-31T10-54-40 ... Done.
step 1 - loss = 5.589, (3.403 sec/step)
step 2 - loss = 5.498, (3.220 sec/step)
step 3 - loss = 5.493, (3.605 sec/step)
step 4 - loss = 5.399, (3.675 sec/step)
step 5 - loss = 5.312, (1.936 sec/step)
step 6 - loss = 5.277, (3.636 sec/step)
step 7 - loss = 5.186, (3.772 sec/step)
step 8 - loss = 5.509, (4.324 sec/step)
step 9 - loss = 5.239, (4.686 sec/step)
step 10 - loss = 5.144, (2.501 sec/step)
step 11 - loss = 4.987, (2.861 sec/step)
step 12 - loss = 4.952, (2.903 sec/step)
step 13 - loss = 4.970, (2.990 sec/step

step 178 - loss = 2.796, (2.508 sec/step)
step 179 - loss = 2.209, (1.559 sec/step)
step 180 - loss = 2.670, (1.560 sec/step)
step 181 - loss = 2.155, (2.435 sec/step)
step 182 - loss = 2.402, (2.651 sec/step)
step 183 - loss = 2.503, (1.479 sec/step)
step 184 - loss = 2.465, (2.372 sec/step)
step 185 - loss = 2.860, (1.443 sec/step)
step 186 - loss = 2.023, (3.556 sec/step)
step 187 - loss = 2.114, (1.693 sec/step)
step 188 - loss = 2.238, (1.933 sec/step)
step 189 - loss = 3.029, (3.501 sec/step)
step 190 - loss = 2.113, (3.246 sec/step)
step 191 - loss = 2.445, (1.723 sec/step)
step 192 - loss = 2.332, (1.854 sec/step)
step 193 - loss = 2.765, (2.029 sec/step)
step 194 - loss = 2.538, (1.665 sec/step)
step 195 - loss = 2.749, (1.798 sec/step)
step 196 - loss = 3.512, (3.293 sec/step)
step 197 - loss = 2.932, (1.488 sec/step)
step 198 - loss = 2.217, (2.409 sec/step)
step 199 - loss = 2.394, (1.654 sec/step)
step 200 - loss = 2.488, (2.968 sec/step)
Storing checkpoint to ./logdir/tra

step 367 - loss = 2.422, (1.508 sec/step)
step 368 - loss = 1.911, (1.968 sec/step)
step 369 - loss = 2.327, (1.626 sec/step)
step 370 - loss = 2.359, (2.534 sec/step)
step 371 - loss = 2.344, (1.851 sec/step)
step 372 - loss = 2.037, (2.106 sec/step)
step 373 - loss = 2.306, (1.424 sec/step)
step 374 - loss = 1.783, (2.489 sec/step)
step 375 - loss = 2.366, (1.572 sec/step)
step 376 - loss = 2.254, (1.775 sec/step)
step 377 - loss = 2.094, (1.602 sec/step)
step 378 - loss = 1.680, (1.314 sec/step)
step 379 - loss = 2.253, (1.558 sec/step)
step 380 - loss = 1.635, (1.850 sec/step)
step 381 - loss = 2.461, (2.006 sec/step)
step 382 - loss = 1.680, (2.513 sec/step)
step 383 - loss = 1.495, (2.096 sec/step)
step 384 - loss = 1.629, (2.958 sec/step)
step 385 - loss = 0.657, (2.525 sec/step)
step 386 - loss = 2.721, (2.201 sec/step)
step 387 - loss = 2.032, (1.961 sec/step)
step 388 - loss = 1.681, (1.985 sec/step)
step 389 - loss = 2.218, (1.842 sec/step)
step 390 - loss = 2.398, (2.163 se

step 556 - loss = 2.201, (2.113 sec/step)
step 557 - loss = 1.906, (1.223 sec/step)
step 558 - loss = 2.263, (1.681 sec/step)
step 559 - loss = 2.430, (1.603 sec/step)
step 560 - loss = 2.311, (1.657 sec/step)
step 561 - loss = 2.116, (1.969 sec/step)
step 562 - loss = 2.103, (1.827 sec/step)
step 563 - loss = 2.266, (2.418 sec/step)
step 564 - loss = 1.591, (1.415 sec/step)
step 565 - loss = 2.785, (1.923 sec/step)
step 566 - loss = 2.534, (1.524 sec/step)
step 567 - loss = 1.705, (2.333 sec/step)
step 568 - loss = 2.168, (3.060 sec/step)
step 569 - loss = 1.900, (1.963 sec/step)
step 570 - loss = 1.925, (2.066 sec/step)
step 571 - loss = 1.811, (4.623 sec/step)
step 572 - loss = 2.050, (2.686 sec/step)
step 573 - loss = 1.544, (1.647 sec/step)
step 574 - loss = 1.941, (2.356 sec/step)
step 575 - loss = 1.819, (1.960 sec/step)
step 576 - loss = 2.187, (1.542 sec/step)
step 577 - loss = 2.136, (2.988 sec/step)
step 578 - loss = 2.263, (1.488 sec/step)
step 579 - loss = 1.765, (2.629 se

step 747 - loss = 2.645, (1.531 sec/step)
step 748 - loss = 2.033, (2.486 sec/step)
step 749 - loss = 0.703, (3.323 sec/step)
step 750 - loss = 1.944, (1.694 sec/step)
Storing checkpoint to ./logdir/train/2018-05-31T10-54-40 ... Done.
step 751 - loss = 2.459, (1.981 sec/step)
step 752 - loss = 1.898, (1.917 sec/step)
step 753 - loss = 1.839, (0.927 sec/step)
step 754 - loss = 2.147, (1.680 sec/step)
step 755 - loss = 2.238, (2.573 sec/step)
step 756 - loss = 2.732, (1.877 sec/step)
step 757 - loss = 2.639, (2.482 sec/step)
step 758 - loss = 1.974, (1.877 sec/step)
step 759 - loss = 2.591, (2.153 sec/step)
step 760 - loss = 1.989, (2.323 sec/step)
step 761 - loss = 1.542, (0.881 sec/step)
step 762 - loss = 2.359, (2.484 sec/step)
step 763 - loss = 2.741, (2.340 sec/step)
step 764 - loss = 2.569, (2.312 sec/step)
step 765 - loss = 2.053, (1.637 sec/step)
step 766 - loss = 1.988, (1.103 sec/step)
step 767 - loss = 2.422, (1.809 sec/step)
step 768 - loss = 1.364, (3.793 sec/step)
step 769 

...

step 10740 - loss = 2.623, (1.450 sec/step)
step 10741 - loss = 1.828, (1.312 sec/step)
step 10742 - loss = 1.848, (1.623 sec/step)
step 10743 - loss = 1.727, (1.099 sec/step)
step 10744 - loss = 2.223, (1.273 sec/step)
step 10745 - loss = 1.795, (1.100 sec/step)
step 10746 - loss = 2.099, (2.957 sec/step)
step 10747 - loss = 2.013, (3.046 sec/step)
step 10748 - loss = 1.801, (1.890 sec/step)
step 10749 - loss = 1.614, (1.099 sec/step)
step 10750 - loss = 2.014, (1.759 sec/step)
Storing checkpoint to ./logdir/train/2018-05-31T10-54-40 ... Done.
step 10751 - loss = 2.032, (1.311 sec/step)
step 10752 - loss = 1.825, (1.710 sec/step)
step 10753 - loss = 1.955, (1.275 sec/step)
step 10754 - loss = 2.269, (2.483 sec/step)
step 10755 - loss = 1.022, (1.453 sec/step)
step 10756 - loss = 1.960, (1.099 sec/step)
step 10757 - loss = 1.488, (1.817 sec/step)
step 10758 - loss = 1.728, (1.706 sec/step)
step 10759 - loss = 2.153, (1.710 sec/step)
step 10760 - loss = 1.625, (0.997 sec/step)
step 1076

step 10921 - loss = 2.027, (1.768 sec/step)
step 10922 - loss = 1.978, (2.478 sec/step)
step 10923 - loss = 1.788, (1.161 sec/step)
step 10924 - loss = 1.469, (2.532 sec/step)
step 10925 - loss = 1.973, (2.474 sec/step)
step 10926 - loss = 1.803, (1.377 sec/step)
step 10927 - loss = 1.800, (1.017 sec/step)
step 10928 - loss = 2.243, (2.363 sec/step)
step 10929 - loss = 1.826, (1.768 sec/step)
step 10930 - loss = 2.014, (1.212 sec/step)
step 10931 - loss = 1.161, (1.254 sec/step)
step 10932 - loss = 1.280, (1.227 sec/step)
step 10933 - loss = 1.773, (2.444 sec/step)
step 10934 - loss = 1.713, (1.198 sec/step)
step 10935 - loss = 1.687, (1.655 sec/step)
step 10936 - loss = 1.999, (1.287 sec/step)
step 10937 - loss = 2.250, (1.974 sec/step)
step 10938 - loss = 1.860, (1.261 sec/step)
step 10939 - loss = 1.715, (1.051 sec/step)
step 10940 - loss = 1.377, (1.330 sec/step)
step 10941 - loss = 1.437, (2.931 sec/step)
step 10942 - loss = 1.823, (1.647 sec/step)
step 10943 - loss = 2.103, (3.01

step 11102 - loss = 2.288, (2.779 sec/step)
step 11103 - loss = 1.904, (2.124 sec/step)
step 11104 - loss = 1.609, (0.969 sec/step)
step 11105 - loss = 1.603, (0.936 sec/step)
step 11106 - loss = 1.321, (1.196 sec/step)
step 11107 - loss = 1.328, (1.589 sec/step)
step 11108 - loss = 1.807, (1.289 sec/step)
step 11109 - loss = 1.867, (1.555 sec/step)
step 11110 - loss = 1.980, (0.987 sec/step)
step 11111 - loss = 2.709, (1.003 sec/step)
step 11112 - loss = 1.653, (1.457 sec/step)
step 11113 - loss = 1.625, (2.747 sec/step)
step 11114 - loss = 1.543, (1.000 sec/step)
step 11115 - loss = 1.827, (1.436 sec/step)
step 11116 - loss = 1.947, (1.601 sec/step)
step 11117 - loss = 1.977, (1.456 sec/step)
step 11118 - loss = 1.754, (1.304 sec/step)
step 11119 - loss = 1.831, (1.615 sec/step)
step 11120 - loss = 1.759, (1.114 sec/step)
step 11121 - loss = 2.268, (1.542 sec/step)
step 11122 - loss = 1.350, (2.194 sec/step)
step 11123 - loss = 1.085, (1.394 sec/step)
step 11124 - loss = 1.675, (0.94

step 11284 - loss = 2.004, (1.542 sec/step)
step 11285 - loss = 1.579, (3.062 sec/step)
step 11286 - loss = 1.408, (1.445 sec/step)
step 11287 - loss = 1.361, (1.292 sec/step)
step 11288 - loss = 1.490, (2.589 sec/step)
step 11289 - loss = 1.420, (1.156 sec/step)
step 11290 - loss = 1.868, (0.858 sec/step)
step 11291 - loss = 1.846, (1.544 sec/step)
step 11292 - loss = 1.698, (1.067 sec/step)
step 11293 - loss = 2.044, (2.738 sec/step)
step 11294 - loss = 1.966, (2.123 sec/step)
step 11295 - loss = 2.002, (1.227 sec/step)
step 11296 - loss = 1.841, (1.345 sec/step)
step 11297 - loss = 1.710, (2.012 sec/step)
step 11298 - loss = 2.245, (3.000 sec/step)
step 11299 - loss = 1.620, (1.187 sec/step)
step 11300 - loss = 1.850, (1.339 sec/step)
Storing checkpoint to ./logdir/train/2018-05-31T10-54-40 ... Done.
step 11301 - loss = 1.684, (1.152 sec/step)
step 11302 - loss = 2.489, (1.372 sec/step)
step 11303 - loss = 2.068, (1.254 sec/step)
step 11304 - loss = 1.760, (1.415 sec/step)
step 1130

step 11465 - loss = 1.482, (1.207 sec/step)
step 11466 - loss = 1.344, (1.291 sec/step)
step 11467 - loss = 1.638, (1.042 sec/step)
step 11468 - loss = 1.811, (0.989 sec/step)
step 11469 - loss = 1.820, (0.856 sec/step)
step 11470 - loss = 2.175, (1.252 sec/step)
step 11471 - loss = 1.453, (1.710 sec/step)
step 11472 - loss = 1.283, (1.031 sec/step)
step 11473 - loss = 1.798, (1.427 sec/step)
step 11474 - loss = 1.396, (1.391 sec/step)
step 11475 - loss = 1.721, (1.145 sec/step)
step 11476 - loss = 1.572, (1.020 sec/step)
step 11477 - loss = 1.985, (2.385 sec/step)
step 11478 - loss = 1.884, (2.041 sec/step)
step 11479 - loss = 2.079, (1.068 sec/step)
step 11480 - loss = 1.555, (2.374 sec/step)
step 11481 - loss = 1.683, (2.486 sec/step)
step 11482 - loss = 0.885, (1.063 sec/step)
step 11483 - loss = 1.830, (1.970 sec/step)
step 11484 - loss = 1.830, (1.395 sec/step)
step 11485 - loss = 1.661, (1.392 sec/step)
step 11486 - loss = 1.703, (1.644 sec/step)
step 11487 - loss = 1.718, (1.84